### Esta celda inicia spark

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc


sc = SparkContext("local[2]", "TwitterDemo")
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
ssc.checkpoint( "file:///Users/Pepe/Desktop/Github/FinalBDNR/Files/checkpoint")

socket_stream = ssc.socketTextStream("localhost", 4040)

lines = socket_stream.window(10)

In [ ]:
ssc.stop()

In [ ]:
type(lines)

In [2]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple('Tweet', fields)

(lines
  .flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("#") )
  .map( lambda word: ( word.lower(), 1 ) )
  .reduceByKey( lambda a, b: a + b )
  .map( lambda rec: Tweet( rec[0], rec[1] ) )
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
  .limit(10).registerTempTable("tweets") ) )

ssc.start()

In [3]:
import matplotlib.pyplot as plt
import seaborn as sn
import time
from IPython import display


count = 0
while count < 10:
    time.sleep(20)
    top_10_tweets = sqlContext.sql( 'SELECT tag, count FROM tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize = (10, 8))
    sn.barplot(x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

ssc.awaitTermination()

<Figure size 1000x800 with 1 Axes>

Py4JJavaError: An error occurred while calling o27.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\Users\Pepe\Anaconda3\lib\site-packages\pyspark\streaming\util.py", line 65, in call
    r = self.func(t, *rdds)
  File "C:\Users\Pepe\Anaconda3\lib\site-packages\pyspark\streaming\dstream.py", line 159, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-2-b4b82511f3c8>", line 11, in <lambda>
    .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
  File "C:\Users\Pepe\Anaconda3\lib\site-packages\pyspark\sql\session.py", line 58, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "C:\Users\Pepe\Anaconda3\lib\site-packages\pyspark\sql\session.py", line 689, in createDataFrame
    rdd, schema = self._createFromRDD(data.map(prepare), schema, samplingRatio)
  File "C:\Users\Pepe\Anaconda3\lib\site-packages\pyspark\sql\session.py", line 384, in _crea

In [ ]:
df = sqlContext.sql("SELECT * FROM tweets")
df.show()